In [7]:
# 03_retrieval_testing.ipynb - Chunk 1: Setup and Embedding Generation

import os
from pathlib import Path
import PyPDF2
import numpy as np
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import uuid
import time

# Import all working functions from previous notebooks
import re
import tiktoken

# Initialize tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

def count_tokens(text):
    """Count tokens in text"""
    return len(tokenizer.encode(text))

def clean_text(text):
    """Basic text cleaning"""
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    text = re.sub(r'([.!?])\s*([A-Z])', r'\1 \2', text)
    text = re.sub(r'([a-z])\s*\n\s*([a-z])', r'\1 \2', text)
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = text.strip()
    return text

def fixed_enhanced_cleaning(text):
    """Enhanced cleaning for pandas documentation"""
    text = clean_text(text)
    
    # Fix specific PDF artifacts
    text = re.sub(r'\bwher\s+e\b', 'where', text)
    text = re.sub(r'\btransfor\s+ms\b', 'transforms', text)
    text = re.sub(r'\bcomp\s+lex\b', 'complex', text)
    text = re.sub(r'\boper\s+ation\s+s\b', 'operations', text)
    text = re.sub(r'\bData\s+Frame\b', 'DataFrame', text)
    text = re.sub(r'\bgroup\s+by\b', 'groupby', text, flags=re.IGNORECASE)
    
    return text

def detect_code_blocks(text):
    """Detect if text contains code examples"""
    code_patterns = [
        r'import\s+\w+', r'pd\.\w+', r'df\.\w+', r'print\s*\(',
        r'=\s*pd\.', r'\.groupby\(', r'\.merge\(', r'\.iloc\[', r'\.loc\['
    ]
    code_score = sum(len(re.findall(pattern, text, re.IGNORECASE)) for pattern in code_patterns)
    return code_score > 2

def robust_text_splitting(text):
    """Split text using multiple strategies"""
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    
    if len(paragraphs) < 3:
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        paragraphs = []
        current_para = ""
        
        for line in lines:
            if line.endswith(('.', '!', '?', ':')) or len(current_para) > 300:
                current_para += " " + line if current_para else line
                if len(current_para.split()) > 20:
                    paragraphs.append(current_para)
                    current_para = ""
            else:
                current_para += " " + line if current_para else line
        
        if current_para:
            paragraphs.append(current_para)
    
    if len(paragraphs) < 2:
        sentences = re.split(r'[.!?]+\s+', text)
        paragraphs = []
        current_para = ""
        
        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue
            if count_tokens(current_para + " " + sentence) > 200:
                if current_para:
                    paragraphs.append(current_para)
                current_para = sentence
            else:
                current_para += " " + sentence if current_para else sentence
        
        if current_para:
            paragraphs.append(current_para)
    
    return paragraphs

def working_chunking_strategy(text, target_size=1000, min_size=400):
    """Final working chunking strategy"""
    cleaned_text = fixed_enhanced_cleaning(text)
    segments = robust_text_splitting(cleaned_text)
    
    chunks = []
    current_chunk = ""
    
    for segment in segments:
        current_tokens = count_tokens(current_chunk)
        segment_tokens = count_tokens(segment)
        
        if current_tokens + segment_tokens > target_size and current_tokens >= min_size:
            chunks.append({
                'text': current_chunk.strip(),
                'token_count': current_tokens,
                'has_code': detect_code_blocks(current_chunk)
            })
            current_chunk = segment
        else:
            current_chunk += "\n\n" + segment if current_chunk else segment
    
    if current_chunk and count_tokens(current_chunk) >= min_size:
        chunks.append({
            'text': current_chunk.strip(),
            'token_count': count_tokens(current_chunk),
            'has_code': detect_code_blocks(current_chunk)
        })
    
    return chunks

# Setup paths and models
PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
PDF_FILE = PROJECT_ROOT / 'data' / 'raw' / 'mastering_pandas_2025.pdf'

# Initialize embedding model
print("Loading embedding model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"Model loaded. Embedding dimension: {embedding_model.get_sentence_embedding_dimension()}")

# Initialize Qdrant client
print("Connecting to Qdrant...")
try:
    qdrant_client = QdrantClient("localhost", port=6333)
    
    # Test connection
    collections = qdrant_client.get_collections()
    print(f"✓ Connected to Qdrant. Existing collections: {len(collections.collections)}")
except Exception as e:
    print(f"✗ Failed to connect to Qdrant: {e}")
    print("Make sure Qdrant is running: docker run -p 6333:6333 qdrant/qdrant")

# Prepare test data - extract and chunk some pages
def extract_all_text(pdf_path, start_page=11, end_page=None):
    """Extract text from content pages"""
    all_text = []
    
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        end_page = end_page or len(pdf_reader.pages)
        
        for page_num in range(start_page, end_page):
            try:
                text = pdf_reader.pages[page_num].extract_text()
                if text.strip():
                    all_text.append({'page': page_num, 'raw_text': text})
            except Exception as e:
                print(f"Error extracting page {page_num}: {e}")
    
    return all_text

# Extract sample pages for testing
print("\nExtracting test pages...")
test_pages = extract_all_text(PDF_FILE, start_page=40, end_page=55)  # 15 pages
combined_text = "\n\n".join([page['raw_text'] for page in test_pages])

print(f"Extracted {len(test_pages)} pages")
print(f"Total text length: {len(combined_text)} characters")

# Test embedding generation
print("\nTesting embedding generation...")
sample_text = "Pandas DataFrame is a powerful data structure for data analysis in Python."
sample_embedding = embedding_model.encode(sample_text)

print(f"Sample text: {sample_text}")
print(f"Embedding shape: {sample_embedding.shape}")
print(f"Embedding preview: {sample_embedding[:5]}")
print("✓ Embedding generation working")

Loading embedding model...
Model loaded. Embedding dimension: 384
Connecting to Qdrant...
Model loaded. Embedding dimension: 384
Connecting to Qdrant...
✓ Connected to Qdrant. Existing collections: 0

Extracting test pages...
✓ Connected to Qdrant. Existing collections: 0

Extracting test pages...
Extracted 15 pages
Total text length: 14561 characters

Testing embedding generation...
Extracted 15 pages
Total text length: 14561 characters

Testing embedding generation...
Sample text: Pandas DataFrame is a powerful data structure for data analysis in Python.
Embedding shape: (384,)
Embedding preview: [-0.01055324 -0.03094391 -0.06620552 -0.04011484  0.03775875]
✓ Embedding generation working
Sample text: Pandas DataFrame is a powerful data structure for data analysis in Python.
Embedding shape: (384,)
Embedding preview: [-0.01055324 -0.03094391 -0.06620552 -0.04011484  0.03775875]
✓ Embedding generation working


In [8]:
# Chunk 2: Create and Store Embeddings

# Chunk the extracted text using our working strategy
print("Chunking extracted text...")
chunks = working_chunking_strategy(combined_text, target_size=1000, min_size=400)

print(f"Created {len(chunks)} chunks")
print(f"Token distribution: {[c['token_count'] for c in chunks]}")
print(f"Code chunks: {sum(1 for c in chunks if c['has_code'])}")

# Generate embeddings for all chunks
print(f"\nGenerating embeddings for {len(chunks)} chunks...")
start_time = time.time()

chunk_embeddings = []
for i, chunk in enumerate(chunks):
    embedding = embedding_model.encode(chunk['text'])
    chunk_embeddings.append(embedding)
    
    if (i + 1) % 5 == 0:
        print(f"  Generated {i + 1}/{len(chunks)} embeddings...")

embedding_time = time.time() - start_time
print(f"✓ Generated all embeddings in {embedding_time:.2f} seconds")
print(f"Average time per chunk: {embedding_time/len(chunks):.3f} seconds")

# Create Qdrant collection
collection_name = "pandas_docs_test"

print(f"\nCreating Qdrant collection: {collection_name}")
try:
    # Delete collection if it exists
    try:
        qdrant_client.delete_collection(collection_name)
        print("  Deleted existing collection")
    except:
        pass
    
    # Create new collection
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)
    )
    print("✓ Collection created successfully")
except Exception as e:
    print(f"✗ Error creating collection: {e}")

# Prepare points for insertion
print("Preparing data points...")
points = []

for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
    point = PointStruct(
        id=str(uuid.uuid4()),
        vector=embedding.tolist(),
        payload={
            "text": chunk['text'],
            "token_count": chunk['token_count'],
            "has_code": chunk['has_code'],
            "chunk_index": i,
            "preview": chunk['text'][:200] + "..." if len(chunk['text']) > 200 else chunk['text']
        }
    )
    points.append(point)

print(f"Prepared {len(points)} points for insertion")

# Insert points into Qdrant
print("Inserting points into Qdrant...")
try:
    result = qdrant_client.upsert(
        collection_name=collection_name,
        points=points
    )
    print(f"✓ Inserted {len(points)} points successfully")
    print(f"Operation result: {result}")
except Exception as e:
    print(f"✗ Error inserting points: {e}")

# Verify insertion
print("\nVerifying insertion...")
try:
    collection_info = qdrant_client.get_collection(collection_name)
    print(f"Collection info: {collection_info}")
    
    # Count points
    count_result = qdrant_client.count(collection_name)
    print(f"✓ Total points in collection: {count_result.count}")
except Exception as e:
    print(f"✗ Error verifying: {e}")

# Test basic similarity search
print("\nTesting basic similarity search...")
test_query = "How to create a pandas DataFrame?"
query_embedding = embedding_model.encode(test_query)

try:
    search_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=3
    )
    
    print(f"Query: '{test_query}'")
    print(f"Found {len(search_results)} results:")
    
    for i, result in enumerate(search_results, 1):
        print(f"\n  Result {i} (Score: {result.score:.4f}):")
        print(f"    Tokens: {result.payload['token_count']}")
        print(f"    Has code: {result.payload['has_code']}")
        print(f"    Preview: {result.payload['preview']}")
        
    print("✓ Basic similarity search working!")
    
except Exception as e:
    print(f"✗ Error in similarity search: {e}")

Chunking extracted text...
Created 4 chunks
Token distribution: [945, 885, 946, 476]
Code chunks: 3

Generating embeddings for 4 chunks...
Created 4 chunks
Token distribution: [945, 885, 946, 476]
Code chunks: 3

Generating embeddings for 4 chunks...
✓ Generated all embeddings in 0.28 seconds
Average time per chunk: 0.070 seconds

Creating Qdrant collection: pandas_docs_test
  Deleted existing collection
✓ Generated all embeddings in 0.28 seconds
Average time per chunk: 0.070 seconds

Creating Qdrant collection: pandas_docs_test
  Deleted existing collection
✓ Collection created successfully
Preparing data points...
Prepared 4 points for insertion
Inserting points into Qdrant...
✓ Inserted 4 points successfully
Operation result: operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>

Verifying insertion...
Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=4 segments_co

C:\Users\MOHAMMED ZAKEER\AppData\Local\Temp\ipykernel_12364\3139400906.py:98: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant_client.search(


In [9]:
# Chunk 3: Query Testing and Evaluation

def test_retrieval_quality(query, expected_content_type=None, top_k=3):
    """Test retrieval quality for a given query"""
    
    print(f"\n{'='*60}")
    print(f"QUERY: {query}")
    print(f"{'='*60}")
    
    # Generate query embedding
    query_embedding = embedding_model.encode(query)
    
    # Search
    search_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=top_k
    )
    
    print(f"Found {len(search_results)} results:")
    
    for i, result in enumerate(search_results, 1):
        score = result.score
        has_code = result.payload['has_code']
        tokens = result.payload['token_count']
        
        # Color coding for scores
        score_status = "🟢" if score > 0.8 else "🟡" if score > 0.6 else "🔴"
        code_status = "💻" if has_code else "📝"
        
        print(f"\n  {score_status} Result {i}: {score:.4f} {code_status}")
        print(f"    Tokens: {tokens} | Has Code: {has_code}")
        print(f"    Text Preview:")
        
        # Show first 300 characters with proper formatting
        preview_text = result.payload['text'][:300]
        for line in preview_text.split('\n')[:4]:  # Show first 4 lines
            if line.strip():
                print(f"      {line.strip()}")
        if len(result.payload['text']) > 300:
            print("      ...")
    
    return search_results

# Test different types of pandas queries
test_queries = [
    # Conceptual questions
    "What is a pandas DataFrame?",
    "Difference between Series and DataFrame",
    
    # How-to questions  
    "How to create a DataFrame?",
    "How to select data from DataFrame?",
    
    # Specific function questions
    "pandas groupby function",
    "DataFrame indexing methods",
    
    # Code-focused questions
    "pandas DataFrame code examples",
    "Series creation syntax",
]

print("Testing different query types...")
print("Legend: 🟢 Excellent (>0.8) | 🟡 Good (>0.6) | 🔴 Poor (<0.6)")
print("        💻 Contains Code | 📝 Text Only")

all_results = {}
for query in test_queries:
    results = test_retrieval_quality(query, top_k=2)
    all_results[query] = results
    time.sleep(0.1)  # Small delay for readability

# Analyze overall retrieval quality
print(f"\n{'='*60}")
print("RETRIEVAL QUALITY ANALYSIS")
print(f"{'='*60}")

total_queries = len(test_queries)
excellent_results = 0
good_results = 0
poor_results = 0

for query, results in all_results.items():
    if results:
        top_score = results[0].score
        if top_score > 0.8:
            excellent_results += 1
        elif top_score > 0.6:
            good_results += 1
        else:
            poor_results += 1

print(f"Query Performance:")
print(f"  🟢 Excellent (>0.8): {excellent_results}/{total_queries} ({excellent_results/total_queries*100:.1f}%)")
print(f"  🟡 Good (>0.6): {good_results}/{total_queries} ({good_results/total_queries*100:.1f}%)")
print(f"  🔴 Poor (<0.6): {poor_results}/{total_queries} ({poor_results/total_queries*100:.1f}%)")

# Test search parameters
print(f"\n{'='*40}")
print("TESTING SEARCH PARAMETERS")
print(f"{'='*40}")

test_query = "How to create pandas DataFrame"
print(f"Query: '{test_query}'")

# Test different top_k values
for k in [1, 3, 5]:
    results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=embedding_model.encode(test_query).tolist(),
        limit=k
    )
    avg_score = sum(r.score for r in results) / len(results) if results else 0
    print(f"  Top-{k}: Avg Score = {avg_score:.4f}")

# Test with score threshold
threshold = 0.7
results_filtered = qdrant_client.search(
    collection_name=collection_name,
    query_vector=embedding_model.encode(test_query).tolist(),
    limit=10,
    score_threshold=threshold
)
print(f"  With threshold >{threshold}: {len(results_filtered)} results")

print(f"\n✓ Retrieval testing complete!")
print(f"✓ Vector database performing well for pandas queries")
print(f"✓ Ready for LLM integration testing")

Testing different query types...
Legend: 🟢 Excellent (>0.8) | 🟡 Good (>0.6) | 🔴 Poor (<0.6)
        💻 Contains Code | 📝 Text Only

QUERY: What is a pandas DataFrame?


C:\Users\MOHAMMED ZAKEER\AppData\Local\Temp\ipykernel_12364\3574076862.py:14: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant_client.search(


Found 2 results:

  🟡 Result 1: 0.6371 📝
    Tokens: 476 | Has Code: False
    Text Preview:
      Each of these series demonstrates a key featur e of Pandas: the ability to customize indices and handle diverse data types With Pandas, you can create Series that mirror the structur e and comple xity of real-world data, giving you the tools to analyze and manipulate data in a way that is both power
      ...

  🔴 Result 2: 0.5855 💻
    Tokens: 945 | Has Code: True
    Text Preview:
      Chapter 4: Data Structures In P andas: Series and Data F rames In Pandas, two fundamental data structures form the backbone of everything you’ll do: Series and Data Frames Think of these as the building blocks for data analysis, allowing you to move from basic data handling to complex manipulation w
      ...

QUERY: Difference between Series and DataFrame
Found 2 results:

  🔴 Result 1: 0.5915 💻
    Tokens: 945 | Has Code: True
    Text Preview:
      Chapter 4: Data Structures In P andas: Series and Da

C:\Users\MOHAMMED ZAKEER\AppData\Local\Temp\ipykernel_12364\3574076862.py:109: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(
C:\Users\MOHAMMED ZAKEER\AppData\Local\Temp\ipykernel_12364\3574076862.py:119: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results_filtered = qdrant_client.search(


In [10]:
# Chunk 4: Test with Fundamental Content (Pages 11-25)

# Extract early chapters that should contain DataFrame basics
print("Extracting fundamental pandas content (pages 11-25)...")
fundamental_pages = extract_all_text(PDF_FILE, start_page=11, end_page=26)  # 15 pages of basics
fundamental_text = "\n\n".join([page['raw_text'] for page in fundamental_pages])

print(f"Extracted {len(fundamental_pages)} fundamental pages")
print(f"Total text length: {len(fundamental_text)} characters")

# Preview what topics we're getting
print(f"\nContent preview (first 500 chars):")
print(fundamental_text[:500])

# Chunk the fundamental content
print(f"\nChunking fundamental content...")
fundamental_chunks = working_chunking_strategy(fundamental_text, target_size=1000, min_size=400)

print(f"Created {len(fundamental_chunks)} chunks from fundamental content")
print(f"Token distribution: {[c['token_count'] for c in fundamental_chunks]}")
print(f"Code chunks: {sum(1 for c in fundamental_chunks if c['has_code'])}")

# Generate embeddings for fundamental chunks
print(f"\nGenerating embeddings for {len(fundamental_chunks)} fundamental chunks...")
start_time = time.time()

fundamental_embeddings = []
for i, chunk in enumerate(fundamental_chunks):
    embedding = embedding_model.encode(chunk['text'])
    fundamental_embeddings.append(embedding)
    
    if (i + 1) % 5 == 0:
        print(f"  Generated {i + 1}/{len(fundamental_chunks)} embeddings...")

embedding_time = time.time() - start_time
print(f"✓ Generated all embeddings in {embedding_time:.2f} seconds")

# Create new collection for fundamental content
collection_name_v2 = "pandas_fundamentals"

print(f"\nCreating new collection: {collection_name_v2}")
try:
    # Delete if exists
    try:
        qdrant_client.delete_collection(collection_name_v2)
        print("  Deleted existing collection")
    except:
        pass
    
    # Create collection
    qdrant_client.create_collection(
        collection_name=collection_name_v2,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)
    )
    print("✓ Fundamental collection created")
except Exception as e:
    print(f"✗ Error: {e}")

# Prepare and insert fundamental points
print("Inserting fundamental content...")
fundamental_points = []

for i, (chunk, embedding) in enumerate(zip(fundamental_chunks, fundamental_embeddings)):
    point = PointStruct(
        id=str(uuid.uuid4()),
        vector=embedding.tolist(),
        payload={
            "text": chunk['text'],
            "token_count": chunk['token_count'],
            "has_code": chunk['has_code'],
            "chunk_index": i,
            "content_type": "fundamental",
            "preview": chunk['text'][:200] + "..." if len(chunk['text']) > 200 else chunk['text']
        }
    )
    fundamental_points.append(point)

try:
    result = qdrant_client.upsert(
        collection_name=collection_name_v2,
        points=fundamental_points
    )
    print(f"✓ Inserted {len(fundamental_points)} fundamental points")
    
    # Verify
    count_result = qdrant_client.count(collection_name_v2)
    print(f"✓ Verified: {count_result.count} points in collection")
    
except Exception as e:
    print(f"✗ Error inserting: {e}")

# Test the same queries on fundamental content
print(f"\n{'='*60}")
print("TESTING QUERIES ON FUNDAMENTAL CONTENT")
print(f"{'='*60}")

# Updated test function to use query_points (avoiding deprecation warning)
def test_fundamental_retrieval(query, top_k=3):
    """Test retrieval on fundamental content"""
    
    print(f"\nQUERY: {query}")
    print(f"-" * 50)
    
    query_embedding = embedding_model.encode(query)
    
    # Using query_points to avoid deprecation warning
    search_results = qdrant_client.query_points(
        collection_name=collection_name_v2,
        query=query_embedding.tolist(),
        limit=top_k
    )
    
    results = search_results.points
    print(f"Found {len(results)} results:")
    
    for i, result in enumerate(results, 1):
        score = result.score
        has_code = result.payload['has_code']
        tokens = result.payload['token_count']
        
        score_status = "🟢" if score > 0.8 else "🟡" if score > 0.6 else "🔴"
        code_status = "💻" if has_code else "📝"
        
        print(f"\n  {score_status} Result {i}: {score:.4f} {code_status}")
        print(f"    Tokens: {tokens} | Has Code: {has_code}")
        
        # Show meaningful preview
        preview_text = result.payload['text'][:400]
        lines = [line.strip() for line in preview_text.split('\n') if line.strip()]
        for line in lines[:3]:  # First 3 meaningful lines
            print(f"      {line}")
        if len(result.payload['text']) > 400:
            print("      ...")
    
    return results

# Test key queries on fundamental content
key_queries = [
    "What is a pandas DataFrame?",
    "How to create a DataFrame?", 
    "Difference between Series and DataFrame",
    "DataFrame basic operations"
]

fundamental_results = {}
for query in key_queries:
    results = test_fundamental_retrieval(query, top_k=2)
    fundamental_results[query] = results

# Compare with previous results
print(f"\n{'='*60}")
print("IMPROVEMENT ANALYSIS")
print(f"{'='*60}")

for query in key_queries:
    if query in all_results and query in fundamental_results:
        old_score = all_results[query][0].score if all_results[query] else 0
        new_score = fundamental_results[query][0].score if fundamental_results[query] else 0
        improvement = new_score - old_score
        
        status = "📈" if improvement > 0.1 else "📊" if improvement > 0 else "📉"
        print(f"{status} '{query}': {old_score:.3f} → {new_score:.3f} ({improvement:+.3f})")

print(f"\n✓ Fundamental content testing complete!")
print(f"✓ Ready to compare and proceed to LLM integration")

Extracting fundamental pandas content (pages 11-25)...
Extracted 15 fundamental pages
Total text length: 18472 characters

Content preview (first 500 chars):
environmental science, data holds the answers, and pandas
is your toolkit to unlock them.
If you’re diving into data science, machine learning, deep
learning, or artiﬁcial intelligence, one library you absolutely
need to know is pandas . Real-world data rarely comes
clean and ready for analysis. Often, it’s messy, inconsistent,
and ﬁlled with gaps. To build powerful, accurate models, the
quality of your data matters as much as the algorithms you
use. And that’s where pandas comes in—a superb too

Chunking fundamental content...
Created 4 chunks from fundamental content
Token distribution: [966, 884, 902, 966]
Code chunks: 0

Generating embeddings for 4 fundamental chunks...
Extracted 15 fundamental pages
Total text length: 18472 characters

Content preview (first 500 chars):
environmental science, data holds the answers, and panda

In [13]:
# Quick Fix: Update to Supported Groq Model
# Chunk 5: LLM Integration Setup

import os
from groq import Groq
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()

# Initialize Groq client
print("Setting up Groq LLM integration...")

# You'll need to set your GROQ_API_KEY in .env file
groq_api_key = os.getenv('GROQ_API_KEY')

if not groq_api_key:
    print("⚠️  GROQ_API_KEY not found in environment")
    print("Please set your API key:")
    groq_api_key = input("Enter your Groq API key: ").strip()

# FIX 1: Initialize groq_client (this was missing!)
try:
    groq_client = Groq(api_key=groq_api_key)
    print("✓ Groq client initialized")
except Exception as e:
    print(f"✗ Error initializing Groq: {e}")

# Test available models to find what's currently supported
supported_models = [
    "llama-3.1-8b-instant",
    "mixtral-8x7b-32768", 
    "gemma-7b-it",
    "llama3-8b-8192",
    "llama3-70b-8192"
]

print("Testing supported Groq models...")
working_model = None

for model in supported_models:
    try:
        test_response = groq_client.chat.completions.create(
            messages=[{"role": "user", "content": "Say 'Working!' if you can process this."}],
            model=model,
            max_tokens=20,
            temperature=0.1
        )
        working_model = model
        print(f"✓ {model}: {test_response.choices[0].message.content}")
        break
    except Exception as e:
        print(f"✗ {model}: {str(e)[:100]}...")

if working_model:
    print(f"\n🎯 Using working model: {working_model}")
    print(f"🔧 Recommended model: {working_model}")
    print("✅ Groq setup complete!")
    
    # FIX 2: Comment out rag_query tests since function not defined in this cell
    print("\n📝 Note: To test RAG pipeline, run previous chunks first to define rag_query function")
    
else:
    print(f"\n❌ No working models found. Check your Groq API key and account status.")

Setting up Groq LLM integration...
✓ Groq client initialized
Testing supported Groq models...
✓ Groq client initialized
Testing supported Groq models...
✓ llama-3.1-8b-instant: Working!

🎯 Using working model: llama-3.1-8b-instant
🔧 Recommended model: llama-3.1-8b-instant
✅ Groq setup complete!

📝 Note: To test RAG pipeline, run previous chunks first to define rag_query function
✓ llama-3.1-8b-instant: Working!

🎯 Using working model: llama-3.1-8b-instant
🔧 Recommended model: llama-3.1-8b-instant
✅ Groq setup complete!

📝 Note: To test RAG pipeline, run previous chunks first to define rag_query function
